In [18]:
import sys 

import numpy as np
import pandas as pd 
import networkx as nx
import math
import sympy as sp
import matplotlib.pyplot as plt

np.set_printoptions(threshold=sys.maxsize)

from sympy import *
from numpy import linalg as LA
from numpy.linalg import matrix_rank

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ="all"

In [3]:
#Establecer directorio de datos

DIR='/Users/alex/Desktop/MatrizAdjSano/MatrizAdjTumor'
FILE='/adjmatrix_UNT.txt'

file='{}{}'.format(DIR,FILE)

print('File directory:{}'.format(file))



File directory:/Users/alex/Desktop/MatrizAdjSano/MatrizAdjTumor/adjmatrix_UNT.txt


In [4]:
data=pd.read_table(file)

In [5]:
data.shape

(16748, 16748)

In [6]:
a=np.diag(np.ones(16748),0)
A=data-a

In [8]:
#Este modulo calcula un valor umbral ("threshold") que promedia todos los valores de las 
# entradas de un data set

# Función convertir a lista el resultado de aplicar .mean() a un DataFrame

def mean_to_list(df):
    arr=[]
    for i in range(0,len(df)):
        a=df.iloc[i]
        arr.append(a)
    return arr

# Función para calcular el promedio de las entradas de una lista 

def cal_prom_en_list(num):
    sum_num=0
    for entrada in num:
        sum_num=sum_num +entrada
        
    prom = sum_num/len(num)
    return prom

# Cálculo del umbral, promediando sobre todas las entradas de la matriz de adyacencia

A_prom_cols=A.mean()
A_mean_list=mean_to_list(A_prom_cols)
cal_prom_en_list(A_mean_list)
threshold=cal_prom_en_list(A_mean_list)

0.033064517362779086

In [10]:
threshold

0.033064517362779086

In [11]:
#Función para contruir la matriz de grado a partir de la de la matriz de adyacencia

def degreeMat_from_Adj(mat,n):
    new_data=mat.iloc[:n,:n]
    
    new_data2=new_data.apply(lambda x:(np.where(x<threshold,0,x)))
    adj_mat=new_data2.apply(lambda x:(np.where(x>threshold,1,x)))
    
    #completar la matriz triangular superior para tener la forma completa de una matriz
    # simétrica
    
    adj_mod=np.matrix(adj_mat.fillna(0))
    adj_mod_trans=adj_mod.transpose()
    simm_mat=adj_mod+adj_mod_trans    #type(simm_mat)=numpy matrix
    
    
    Adj_to_nx=nx.from_numpy_matrix(simm_mat)
    L_from_Adj=nx.laplacian_matrix(Adj_to_nx) #obtener la Laplaciana asociada a la de adyacencia
    
    L_arr=L_from_Adj.toarray()
    
    L_np=np.matrix(L_arr)
    
    deg_Mat=L_np+simm_mat
    
    return deg_Mat

In [12]:
# El siguiente módulo, crea un array con los primeros "n" genes que deseen obtener del
# Data Frame "data"

#Parametros: data , n 

def get_genes(data,n):
    
    genes_arr = sorted(data) #obtener un array que contiene los elementos del header del DataFrame
    num_genes_extracted = range(n)
    genes = np.array(genes_arr)[num_genes_extracted]
    
    return genes

In [13]:
#La siguiente función sirve para obtener los elementos de la diagonal de la matriz de
# grado y la guarda en un array llamado out_p


def get_elem_in_diag(Mat):
    Mat_mod = np.squeeze(np.asarray(Mat))#convierte numpy matrix en array
    out_p=[]
    for elem in np.nditer(Mat_mod): #nditer itera sobre arrays
        if elem != 0: # omite los valores 0 en el array pues la matriz de grado fuera de la diagonal tiene solo ceros
            out_p.append(int(elem))
            
    return out_p 

In [14]:
#Función para obtener el número mayor de una lista de números dentro de un array

# El parámetro debe tener el mismo número que la dimensión de la matriz 
# de grado que se está analizando

#def get_biggest_num_in_diag(l,n):
#    count=0
#    for i in range(n):
#        for j in range(n):
#            if l[j]>l[i]:
#                count=l[j]
#    return count   

# En la siguiente función el parámetro "a" es un array que contiene los elementos de 
# la diagonal de la matriz de grado y "N" es el número de elementos que se tomará para
# ordenarlos de mayor a menor.

# El output indica la posición de la entrada y no su valor, pues con ello podremos 
# identificar cuál es el gen que tiene más aristas incidentes dentro de él 

def func(a,N):
    return np.argsort(a)[::-1][:N]

In [16]:
func(get_elem_in_diag(degreeMat_from_Adj(A,100)),100)

array([24, 61, 83,  6, 60, 31, 86, 14, 47, 79, 53, 90, 39, 92, 44, 94, 82,
       34, 57, 64, 17, 97, 13, 36, 29, 77, 78, 27,  3, 41, 32, 18, 48,  0,
       87, 72,  4, 10, 12, 75, 38, 95, 46, 21, 22, 85, 45, 59, 28, 42, 11,
       96, 43, 81,  9, 68, 91, 56, 71, 69, 51, 37, 66, 70, 62, 15, 23, 30,
       99, 33, 98, 84, 58, 16, 55, 89, 49,  8, 20,  1, 63, 50, 93, 74, 73,
        7, 76,  5, 88, 65, 52, 25, 40, 80, 19, 26, 54, 67,  2, 35])

In [15]:
func(get_elem_in_diag(degreeMat_from_Adj(A,1000)),1000)

array([465, 221, 555, 365, 732, 373, 346, 514,  66, 166, 557, 358, 251,
       214, 649, 894, 317,  41, 811, 375, 918, 263, 640, 806, 153, 619,
        31, 565, 297, 127,  43, 520, 728, 241, 690, 747, 371, 398,  47,
       820, 316, 197,  94, 652, 156,  61, 455, 788, 123, 171, 161, 710,
        79, 763, 571, 289, 198, 586, 258, 863, 424, 639, 752, 205, 545,
       415, 892, 641,   3, 165, 441, 862, 479, 868, 132, 593, 861, 549,
       734, 389, 716, 803, 874, 799, 204, 845,  74, 217, 357, 667, 147,
        29, 815, 929, 839, 329, 943, 920, 288,  83, 150, 661, 588,  75,
       886, 883, 805, 272, 416, 426, 663, 534, 470, 266, 719, 891, 481,
       843, 847,  53, 781, 162, 852, 896, 574, 611, 249, 818, 561, 253,
         0, 889, 160, 438, 991, 973, 448, 286,  82, 701, 237, 628, 854,
       875, 392, 767, 445, 882,  24,  48, 633, 212, 718, 686, 469, 798,
       341, 990, 695, 927, 254,  71, 459, 961, 271, 944, 169, 864, 804,
       427, 853, 616,  12, 617, 490, 282, 758, 179, 871, 432, 13

In [19]:
func(get_elem_in_diag(degreeMat_from_Adj(A,2000)),2000)

array([ 465, 1716,  365, 1540,  166, 1467,  358, 1430, 1299, 1631, 1488,
         66,  251,  514, 1855,  263, 1884,  221,  894,  317,  806, 1924,
       1342,  555, 1521,  214, 1741,  811, 1419,  649, 1885,   31, 1128,
        732, 1302, 1899,  415,  127, 1655,  557, 1911,  346, 1363, 1284,
        565, 1620, 1835, 1255,  316, 1935,  918,  619,  805, 1877,  892,
        375, 1396,  929, 1629,  373,  852, 1248, 1116, 1376, 1025, 1868,
       1913, 1644, 1546, 1409,  863,   43, 1514, 1990, 1453, 1170, 1839,
        123,  815,  728, 1358,  197,  156, 1659,  258, 1186, 1541, 1272,
       1092,   41, 1256,  640,   61, 1852,  862, 1591, 1079, 1489, 1804,
       1704, 1461, 1875, 1715, 1225, 1674, 1310, 1970, 1000, 1725,  204,
       1131,  820, 1907,  891, 1031,  455,  652,  690, 1567,   94,  165,
        763, 1860,  138, 1565,  734,  667,   48,  798,  716, 1205, 1830,
       1015, 1737,  107,  788,  371, 1999,   47, 1395,  799,  161,  586,
       1931,  147, 1977, 1285, 1431,  718,   79,  4

In [20]:
func(get_elem_in_diag(degreeMat_from_Adj(A,3000)),3000)

array([ 465, 2457, 2005, 1716, 2096,  365, 2285,  166, 1631, 1467, 1299,
       2077, 2032, 2071, 1924, 1430,  894,  358, 1540, 2597, 1128,  811,
       2054,   66,  514,  251, 1884, 1302, 2517, 1855, 1521,  214, 1741,
       1488,  127, 1255,  806, 1885,   31,  263,  221, 2568, 2263, 1620,
        565, 2504,  317, 1935,  555, 2161, 1655, 1911, 2578,  892, 2124,
       1342,  316, 2453, 2182, 1363, 1419, 2985, 2838, 1839, 1284,  918,
        346, 1835,  415, 2180, 2929, 2079, 2847, 2370,  373, 1546,  891,
       1877,  123,  649, 1025,  805,   61,   48, 2723, 1248, 1031, 2871,
         43, 2454,  732,  197, 2282, 1629, 2575, 1395, 1868, 1830,  557,
       1256, 1725, 1804,  619, 2083, 2034, 1396, 1899, 2313, 2031, 2951,
        863, 1644,  156, 2746, 1358,  852, 2382, 2754, 1720, 1409,  690,
       2442, 1285, 2275, 1170, 1092, 2718, 2065,  375, 1453, 1376, 1674,
        426, 2093, 2044, 2523,   94,  820, 1567,  640, 1913, 1150, 2790,
       2866,  798, 2086, 1541, 2494,  747,  728, 18

In [21]:
func(get_elem_in_diag(degreeMat_from_Adj(A,4000)),4000)

array([ 465, 3742,  365, 3766, 2457, 3866, 1540,  894, 3067, 3383, 2285,
       3241, 1631, 1467, 3976, 2096, 2032, 2005, 3692,  166, 1716, 3384,
       1299, 3779, 2071, 3181, 1430,  263, 1488,  221, 2517, 1128,   66,
       3741,  514, 2077,  251, 1302, 3719, 1255, 3743,  358, 2504, 2597,
        214, 2568,   31, 1884, 3491,  811,  555, 1521, 3355, 1924, 2263,
       3755,  649,  732, 3975, 2985, 1855, 1655, 2054, 2871,  806, 3186,
       1342, 3379, 2161,  316, 1419, 1885, 2838, 2370,  373,  565, 1741,
        619, 1620, 3323,   43, 1935, 1284, 1644,  127,  918, 3235, 3348,
       3673, 3859,  317, 1839, 3923, 3460, 1835, 2180, 1363,  156, 2124,
       3622,  929, 2578, 1256, 3695, 1025, 3343,  557,  891, 1674,  415,
       2182,  123, 1546, 2929, 1376, 1629, 3819, 3458, 3592, 1911, 1877,
       3723, 1514, 1804, 2494, 3977, 3857, 2282, 3815,  892, 2453, 3342,
       2034,  346, 3222, 2275, 3703, 1725, 1000, 3216,  205,  728,   61,
       1409, 1031, 1720, 1453, 1737, 3993, 3700, 35

In [22]:
func(get_elem_in_diag(degreeMat_from_Adj(A,5000)),5000)

array([ 465, 3742,  365, 3766, 3866, 2457, 3241, 1540,  894, 3383, 4617,
       1631, 3067, 3976, 4625, 1716,  166, 3692, 2285, 1467, 1299, 2032,
       2071, 2005,  263, 4336, 2517, 4173,   66, 3181, 2096, 3384,  221,
       4204, 3779, 1430, 4709, 1255, 3719, 1128,  358,   31, 4459,  251,
       4162, 2504, 4230, 4098, 2597,  514, 2077, 3741,  555, 1488, 4407,
        811, 1924, 4835,  214, 4514, 3743, 2568, 3975, 4256,  732, 3755,
       4681, 1884, 4009, 4066, 2370, 1302, 1419, 3491,  619, 2180, 4907,
       1342, 4221,  316, 1284, 4399, 4721, 2985, 1655,  565, 1855, 2263,
         43,  649, 1620, 3355,  806, 1885, 3379, 2871, 1409, 1935, 3323,
       4770, 3859, 1376,  373, 3460,  918, 2161, 2578, 1521, 4001, 4473,
        127, 3343, 3186, 1025, 2054, 1741, 3923, 1629, 2838, 2124, 4485,
       2182, 1839,  929, 4243, 3458, 1835,  891, 4022, 4771, 1256, 4097,
        557, 1363, 3700,  317, 3673, 3977, 3235, 3815, 4429, 3819, 3592,
       3993,  375, 3622, 3509,  156, 1644, 1877, 29

In [23]:
func(get_elem_in_diag(degreeMat_from_Adj(A,6000)),6000)

array([ 465, 3742,  365, 3766, 5436, 3866, 2457, 1631, 3241, 3067, 3976,
       1540, 5971, 1467, 3383, 4617, 3692,  894, 4625, 4336, 1299, 2285,
       3181, 3779, 2005, 1716, 2032, 5576, 1430, 4204, 5630, 2517,  166,
        514, 4407,   66, 4230, 2071, 5563, 2096, 4173, 1255, 4162,  263,
       5632, 4709, 4459,   31, 3741, 4514, 5593, 5763, 3384, 3719, 2597,
       2180, 2077, 3975,  251, 1128, 4098,  555,  358,  221, 4835, 1488,
       2504, 5566, 5817,  811, 5366, 5978, 2568,  806,  214, 3743,  918,
       1302, 3755, 1924, 4681, 2985, 4066, 4721, 4221, 3491, 1620, 1342,
       2370, 1655, 4399, 5623, 1884, 4009, 4907,  619,  649, 1419,   43,
       1885, 5602, 5733, 1363,  732, 4256,  565, 5534, 4770, 1521, 4001,
       3343, 3379, 4485, 2124,  316, 5232, 1935, 3923, 4473, 1839, 1409,
       1376, 3460, 4243,  718,  317, 1644,  557, 3355, 2578, 5488, 1284,
       5040,  127, 2263, 3592, 4022, 1025, 5255, 3977, 4055, 4386, 1855,
       3323, 1629, 3859, 3186, 2838, 2182,  891,  3

In [21]:
func(get_elem_in_diag(degreeMat_from_Adj(A,7000)),7000)

array([ 465, 3742,  365, ..., 5860, 2112, 5143])

In [22]:
func(get_elem_in_diag(degreeMat_from_Adj(A,8000)),8000)

array([ 465, 7948, 3742, ..., 2112, 7635, 5143])

In [23]:
func(get_elem_in_diag(degreeMat_from_Adj(A,9000)),9000)

array([ 465, 7948, 3742, ..., 2112, 5143, 8300])

In [24]:
func(get_elem_in_diag(degreeMat_from_Adj(A,10000)),10000)

array([ 465, 7948, 7261, ..., 5143, 9116, 8300])

In [13]:
get_genes(data,16748)[24]

'ENSG00000003056'

In [14]:
A.mean()

ENSG00000000003    0.000000
ENSG00000000419    0.011007
ENSG00000000457    0.021131
ENSG00000000460    0.064793
ENSG00000000938    0.029265
                     ...   
hsa-mir-9500       0.028380
hsa-mir-96         0.034278
hsa-mir-98         0.031001
hsa-mir-99a        0.032875
hsa-mir-99b        0.032385
Length: 16748, dtype: float64

In [15]:
 func(get_elem_in_diag(degreeMat_from_Adj(A,2000)),2000)

array([ 465, 1716,  365, ..., 1910, 1847,  414])

In [17]:
 func(get_elem_in_diag(degreeMat_from_Adj(A,100)),100)

array([24, 61, 83,  6, 60, 31, 86, 14, 47, 79, 53, 90, 39, 92, 44, 94, 82,
       34, 57, 64, 17, 97, 13, 36, 29, 77, 78, 27,  3, 41, 32, 18, 48,  0,
       87, 72,  4, 10, 12, 75, 38, 95, 46, 21, 22, 85, 45, 59, 28, 42, 11,
       96, 43, 81,  9, 68, 91, 56, 71, 69, 51, 37, 66, 70, 62, 15, 23, 30,
       99, 33, 98, 84, 58, 16, 55, 89, 49,  8, 20,  1, 63, 50, 93, 74, 73,
        7, 76,  5, 88, 65, 52, 25, 40, 80, 19, 26, 54, 67,  2, 35])

In [ ]:
 func(get_elem_in_diag(degreeMat_from_Adj(A,9000)),9000)

In [1]:
 func(get_elem_in_diag(degreeMat_from_Adj(A,2000)),2000)

NameError: name 'func' is not defined